In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [4]:
## get the variables

PATH_1=os.path.join('..','..','..','Weather_Data','ERA5','2018','era_2018.nc')
humidity_data = xr.open_dataset(PATH_1) ## variables
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2018-01-01 ... 2018-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
    expver          (valid_time) <U4 47kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:00 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
humidity_data= humidity_data.drop_vars('number') ## not sure what these two are
humidity_data= humidity_data.drop_vars('expver')
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 23kB 2018-01-01 ... 2018-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:00 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
humidity_data.to_dataframe()

r         q
valid_time          pressure_level latitude longitude                     
2018-01-01 00:00:00 1000.0         49.25    -124.85    93.149689  0.003479
                                            -124.60    95.509064  0.003427
                                            -124.35    94.630157  0.003387
                                            -124.10    85.204376  0.003236
                                            -123.85    73.667267  0.003020
...                                                          ...       ...
2018-12-31 21:00:00 1000.0         24.50    -68.10     60.202324  0.010450
                                            -67.85     60.270683  0.010452
                                            -67.60     60.907402  0.010528
                                            -67.35     62.007011  0.010636
                                            -67.10     63.032402  0.010735

[67744000 rows x 2 columns]

In [7]:
## resample the data to 1 year average

year_avg = humidity_data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2018-12-31
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:00 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
year_avg.to_dataframe()

r         q
valid_time pressure_level latitude longitude                     
2018-12-31 1000.0         49.25    -124.85    82.617592  0.006396
                                   -124.60    79.801094  0.006386
                                   -124.35    77.407349  0.006266
                                   -124.10    75.205780  0.006166
                                   -123.85    72.334999  0.006171
...                                                 ...       ...
                          24.50    -68.10     77.387497  0.014393
                                   -67.85     77.377602  0.014376
                                   -67.60     77.412109  0.014362
                                   -67.35     77.455193  0.014349
                                   -67.10     77.497696  0.014337

[23200 rows x 2 columns]

In [9]:
year_avg = year_avg.squeeze(dim="pressure_level") ## drop the pressure level dimension
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, latitude: 100, longitude: 232)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2018-12-31
Data variables:
    r               (valid_time, latitude, longitude) float32 93kB 82.62 ... ...
    q               (valid_time, latitude, longitude) float32 93kB 0.006396 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:00 GRIB to CDM+CF via cfgrib-0.9.1...

In [10]:
SHAPE_PATH=os.path.join('..','..','..','Shapefiles','county_shapefiles','2018_county_shapefile','cb_2018_us_county_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,"POLYGON ((-89.18137 37.04630, -89.17938 37.053..."
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,"POLYGON ((-84.44266 38.28324, -84.44114 38.283..."
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,"POLYGON ((-86.94486 37.07341, -86.94346 37.074..."
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,"POLYGON ((-84.12662 37.64540, -84.12483 37.646..."
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,"POLYGON ((-83.98428 38.44549, -83.98246 38.450..."
...,...,...,...,...,...,...,...,...,...,...
3228,31,073,00835858,0500000US31073,31073,Gosper,06,1186616237,11831826,"POLYGON ((-100.09510 40.43866, -100.08937 40.4..."
3229,39,075,01074050,0500000US39075,39075,Holmes,06,1094405866,3695230,"POLYGON ((-82.22066 40.66758, -82.19327 40.667..."
3230,48,171,01383871,0500000US48171,48171,Gillespie,06,2740719114,9012764,"POLYGON ((-99.30400 30.49983, -99.28234 30.499..."
3231,55,079,01581100,0500000US55079,55079,Milwaukee,06,625440563,2455383635,"POLYGON ((-88.06959 42.86726, -88.06959 42.872..."


In [11]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-89.18137 37.04630, -89.17938 37.053...
1       POLYGON ((-84.44266 38.28324, -84.44114 38.283...
2       POLYGON ((-86.94486 37.07341, -86.94346 37.074...
3       POLYGON ((-84.12662 37.64540, -84.12483 37.646...
4       POLYGON ((-83.98428 38.44549, -83.98246 38.450...
                              ...                        
3228    POLYGON ((-100.09510 40.43866, -100.08937 40.4...
3229    POLYGON ((-82.22066 40.66758, -82.19327 40.667...
3230    POLYGON ((-99.30400 30.49983, -99.28234 30.499...
3231    POLYGON ((-88.06959 42.86726, -88.06959 42.872...
3232    POLYGON ((-86.26432 43.11830, -86.25103 43.118...
Name: geometry, Length: 3233, dtype: geometry

In [12]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0   -89.181369  37.046305
     1   -89.179384  37.053012
     2   -89.175725  37.062069
     3   -89.171881  37.068184
     4   -89.168087  37.074218
...             ...        ...
3232 148 -86.256651  43.087448
     149 -86.258404  43.090981
     150 -86.261183  43.096580
     151 -86.271996  43.118365
     152 -86.264321  43.118297

[1042614 rows x 2 columns]

In [13]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the values of the variables interpolated in the county.
    ''' 
    longitude= lat_lon.loc[(var), 'x']  # get the longitude 
    extract_val=len(longitude)        ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    
    latitude= lat_lon.loc[(var), 'y']   # get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
    year_avg_finer
    
    df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension, note that this will appear as a column
                                                  ## after removed

    ## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=df.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)

    initial_df=pd.concat(get_vals)
    initial_df=initial_df.drop('valid_time', axis=1) ## remove the valid_time column 
    
    mean=initial_df.mean() ## find the average of the 100 (or less) values
    average_df = pd.DataFrame(mean).T 

    county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
    county_df
    
    return county_df

In [14]:
%%time

## include all counties

df_list=[]

for i in range(0,3233): ## A total of 3232 counties. index in shape file starts from 0 and ends in 3232
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 6min 1s
Wall time: 6min 10s


In [15]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df = final_df.drop('pressure_level', axis=1)
final_df

,r,q
0,67.969326,0.008823
1,73.097803,0.008557
2,70.535708,0.008971
3,73.049503,0.008762
4,73.720388,0.008594
...,...,...
3228,65.646809,0.006781
3229,72.801015,0.00761
3230,66.700521,0.010187
3231,72.295186,0.006402


In [16]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

STATEFP       object
COUNTYFP      object
COUNTYNS      object
AFFGEOID      object
GEOID         object
NAME          object
LSAD          object
ALAND          int64
AWATER         int64
geometry    geometry
fips           int32
dtype: object

In [17]:
county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','AFFGEOID','GEOID','LSAD','ALAND','AWATER'], axis=1)
county_gdf

,NAME,geometry,fips
0,Ballard,"POLYGON ((-89.18137 37.04630, -89.17938 37.053...",21007
1,Bourbon,"POLYGON ((-84.44266 38.28324, -84.44114 38.283...",21017
2,Butler,"POLYGON ((-86.94486 37.07341, -86.94346 37.074...",21031
3,Estill,"POLYGON ((-84.12662 37.64540, -84.12483 37.646...",21065
4,Fleming,"POLYGON ((-83.98428 38.44549, -83.98246 38.450...",21069
...,...,...,...
3228,Gosper,"POLYGON ((-100.09510 40.43866, -100.08937 40.4...",31073
3229,Holmes,"POLYGON ((-82.22066 40.66758, -82.19327 40.667...",39075
3230,Gillespie,"POLYGON ((-99.30400 30.49983, -99.28234 30.499...",48171
3231,Milwaukee,"POLYGON ((-88.06959 42.86726, -88.06959 42.872...",55079


In [18]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,r,q
0,Ballard,"POLYGON ((-89.18137 37.04630, -89.17938 37.053...",21007,67.969326,0.008823
1,Bourbon,"POLYGON ((-84.44266 38.28324, -84.44114 38.283...",21017,73.097803,0.008557
2,Butler,"POLYGON ((-86.94486 37.07341, -86.94346 37.074...",21031,70.535708,0.008971
3,Estill,"POLYGON ((-84.12662 37.64540, -84.12483 37.646...",21065,73.049503,0.008762
4,Fleming,"POLYGON ((-83.98428 38.44549, -83.98246 38.450...",21069,73.720388,0.008594
...,...,...,...,...,...
3228,Gosper,"POLYGON ((-100.09510 40.43866, -100.08937 40.4...",31073,65.646809,0.006781
3229,Holmes,"POLYGON ((-82.22066 40.66758, -82.19327 40.667...",39075,72.801015,0.00761
3230,Gillespie,"POLYGON ((-99.30400 30.49983, -99.28234 30.499...",48171,66.700521,0.010187
3231,Milwaukee,"POLYGON ((-88.06959 42.86726, -88.06959 42.872...",55079,72.295186,0.006402


In [19]:
county_df=merged_df.dropna()
county_df

,NAME,geometry,fips,r,q
0,Ballard,"POLYGON ((-89.18137 37.04630, -89.17938 37.053...",21007,67.969326,0.008823
1,Bourbon,"POLYGON ((-84.44266 38.28324, -84.44114 38.283...",21017,73.097803,0.008557
2,Butler,"POLYGON ((-86.94486 37.07341, -86.94346 37.074...",21031,70.535708,0.008971
3,Estill,"POLYGON ((-84.12662 37.64540, -84.12483 37.646...",21065,73.049503,0.008762
4,Fleming,"POLYGON ((-83.98428 38.44549, -83.98246 38.450...",21069,73.720388,0.008594
...,...,...,...,...,...
3228,Gosper,"POLYGON ((-100.09510 40.43866, -100.08937 40.4...",31073,65.646809,0.006781
3229,Holmes,"POLYGON ((-82.22066 40.66758, -82.19327 40.667...",39075,72.801015,0.00761
3230,Gillespie,"POLYGON ((-99.30400 30.49983, -99.28234 30.499...",48171,66.700521,0.010187
3231,Milwaukee,"POLYGON ((-88.06959 42.86726, -88.06959 42.872...",55079,72.295186,0.006402


In [20]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [21]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,r,q
0,Ballard,21007,67.969325604550335,0.008822649409526
1,Bourbon,21017,73.097803481309626,0.008557260713539
2,Butler,21031,70.535707763459115,0.008970901009193
3,Estill,21065,73.049503371640796,0.008761705523902
4,Fleming,21069,73.720387817021958,0.008593997635594
...,...,...,...,...
3228,Gosper,31073,65.646809085569203,0.006781088406347
3229,Holmes,39075,72.801014760254915,0.007610272217790
3230,Gillespie,48171,66.700520812396420,0.010186873975471
3231,Milwaukee,55079,72.295185724371251,0.006401972430720


In [22]:
county_var = county_var.sort_values(by='fips')
county_var

,NAME,fips,r,q
16,Autauga,1001,70.394067320878662,0.010635091274945
2758,Baldwin,1003,74.858418983174730,0.012255739679584
17,Barbour,1005,73.030934716648176,0.011129821794502
2339,Bibb,1007,71.337635517786481,0.010510536146296
1020,Blount,1009,70.598101544389962,0.010002530006922
...,...,...,...,...
2220,Sweetwater,56037,53.931898520121599,0.003654165217354
1066,Teton,56039,67.441275872250714,0.003732081067021
1539,Uinta,56041,55.200366949635061,0.003632584835703
2798,Washakie,56043,57.113800269274101,0.004245637652218


In [23]:
county_var.to_pickle('era_2018.pkl')